In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import sys
sys.path.append('../')
sys.path.append('../toolbench/inference')
from toolbench.inference.LLM.tool_llama_model import ToolLLaMA
import argparse
import os
sys.argv = ['']
from toolbench.utils import (
    standardize,
    change_name,
    replace_llama_with_condense
)
from toolbench.inference.utils import SimpleChatIO, generate_stream, react_parser
from toolbench.inference.Downstream_tasks.rapidapi import pipeline_runner, rapidapi_wrapper
import json
import copy
from tqdm import tqdm

from transformers import GenerationConfig
import re

In [3]:
data = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final.json', 'r'))

In [4]:
source_max_seq_length = 4096
max_seq_length = 8192
ratio = int(max_seq_length/source_max_seq_length)
replace_llama_with_condense(ratio=ratio)
model = ToolLLaMA('/data4/polyakov/instruction_tuning/data/ToolLLaMA-2-7b-v2', max_sequence_length=max_seq_length)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embeddings from 8192 to 4096
Condensing Positional embedding

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
def parse_model_output(text):
    thought, action, action_input = react_parser(text)
    message = {
        "role": "assistant",
        "content": thought,
        "function_call": {
            "name": action,
            "arguments": action_input
        }
    }
    return message

result = []
for item_idx, item in enumerate(tqdm(data)):
    res_item = copy.deepcopy(item)
    res_item['predictions'] = []
    chain = copy.deepcopy(item['conversations'])
    for step in chain:
        step['role'] = step['from']
        step['content'] = step['value']
        del step['from']
        del step['value']
    for i, step in enumerate(chain):
        if step['role'] == 'assistant':
            thought, action, action_input = react_parser(step['content'])
            if action != 'Finish':
                gt_message = {
                    "role": "assistant",
                    "content": thought,
                    "function_call": {
                        "name": action,
                        "arguments": action_input
                    }
                }
                model.change_messages(chain[:i])
                prompt = model.return_prompt(None, 0)
                pred_messages = model.prediction_non_stream(prompt, num_return_seqs=2, temp=1.5, 
                                                            top_k=100, penalty_alpha=10.0)
                pred_messages = [parse_model_output(message) for message in pred_messages]
                res_item['predictions'].append({'chain_idx': i,
                                                'ground_truth': gt_message,
                                                'prediction': pred_messages}) 
                print(res_item['predictions'][-1])
    result.append(res_item)
    if item_idx > 0 and item_idx % 100 == 0:
        json.dump(result, open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors.json', 'w'))       

  0%|                                                                      | 0/12482 [00:00<?, ?it/s]/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


{'chain_idx': 2, 'ground_truth': {'role': 'assistant', 'content': "I am calling the Chuck Norris API to generate a random joke about Chuck Norris in response to the user's request. This will provide an amusing and entertaining response for the user.", 'function_call': {'name': 'jokes_random_for_chuck_norris', 'arguments': '{}'}}, 'prediction': [{'role': 'assistant', 'content': "Based on the previous conversation, the user wants a random Chuck Norris joke and a fascinating mathematical property about the number 1729. Since the user specifically asked for a Chuck Norris joke, it makes sense to call the 'jokes_random_for_chuck_norris' API to fetch a random joke. As for the fascinating mathematical property about the number 1729, I will need to call the 'numbers' API to get the desired result. I'll use the 'number_property' function and pass the number 1729 as an argument. Hopefully, I can provide the user with the requested Chuck Norris joke and the interesting mathematical property about

  0%|                                                          | 1/12482 [01:05<226:50:06, 65.43s/it]

{'chain_idx': 4, 'ground_truth': {'role': 'assistant', 'content': "I am calling the 'get_math_fact_for_numbers' API with the argument {'number': '1729'}. This API call is made to provide the user with a fascinating mathematical property about the number 1729. By retrieving this information, I can fulfill the user's request for an interesting fact about the number.", 'function_call': {'name': 'get_math_fact_for_numbers', 'arguments': '{\n  "number": "1729"\n}'}}, 'prediction': [{'role': 'assistant', 'content': 'k Norris API was called to fulfill the user\'s request for a random Chuck Norris joke. By generating a response with an entertaining joke about Chuck Norris, I can provide an amusing experience for ]; but also need to address the user\'s question about the number 1729. I will proceed by calling the API \'get_math_fact_for_numbers\' with the argument \'number\' as "1729". This will provide an interesting mathematical property about the requested number, satisfying the user\'s quer

  0%|                                                          | 2/12482 [01:48<180:29:29, 52.06s/it]

{'chain_idx': 4, 'ground_truth': {'role': 'assistant', 'content': 'The service status of the card number validation API is currently "OK". I can proceed with recommending this API to validate card numbers.', 'function_call': {'name': 'validate_card_number_for_luhn_algorithm', 'arguments': '{\n  "number": "4111111111111111"\n}'}}, 'prediction': [{'role': 'assistant', 'content': 'Since I\'m working on a project about credit card fraud detection, I should recommend an API that can validate card numbers. To determine the service status of the API, I called the "service_status_for_luhn_algorithm" function, which doesn\'t require any additional parameters since the arguments are \'{}\'. By checking the service status, I can ensure that the API is operational before I recommend it for use. This step is crucial to ensure reliability in my project.', 'function_call': {'name': 'validate_card_number_for_luhn_algorithm', 'arguments': '{\n  "number": "168200194718824"\n}'}}, {'role': 'assistant', '

  0%|                                                          | 2/12482 [01:54<198:53:08, 57.37s/it]


KeyboardInterrupt: 

In [12]:
result

[{'id': 'Step 7: Can you provide me with a random Chuck Norris joke? I also want to know a fascinating mathematical property about the number 1729. Please find something interesting about it.',
  'conversations': [{'from': 'system',
    'value': 'You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can\'t go back to one of the former state, if you want to restart the task, say "I give 

In [6]:
len(result)

620